# W1M3 - ETL 프로세스 구현하기

--- 

#### 학습 목표
웹사이트에서 데이터를 가져와서 요구사항에 맞게 가공하는 ETL 파이프라인을 만듭니다.
- Web Scraping에 대한 이해
- Pandas DataFrame에 대한 이해
- ETL Process에 대한 이해
- Database & SQL 기초



#### 사전지식
##### 시나리오
- 당신은 해외로 사업을 확장하고자 하는 기업에서 Data Engineer로 일하고 있습니다. 경영진에서 GDP가 높은 국가들을 대상으로 사업성을 평가하려고 합니다.
- 이 자료는 앞으로 경영진에서 지속적으로 요구할 것으로 생각되기 때문에 자동화된 스크립트를 만들어야 합니다.

##### 기능요구사항
- IMF에서 제공하는 국가별 GDP를 구하세요. (https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29)
- 국가별 GDP를 확인할 수 있는 테이블을 만드세요.
- 해당 테이블에는 GDP가 높은 국가들이 먼저 나와야 합니다.
- GDP의 단위는 1B USD이어야 하고 소수점 2자리까지만 표시해 주세요.
- IMF에서 매년 2회 이 자료를 제공하기 때문에 정보가 갱신되더라도 해당 코드를 재사용해서 정보를 얻을 수 있어야 합니다.



#### 화면 출력
- GDP가 100B USD이상이 되는 국가만을 구해서 화면에 출력해야 합니다.
- 각 Region별로 top5 국가의 GDP 평균을 구해서 화면에 출력해야 합니다.



#### 프로그래밍 요구사항
##### 라이브러리 사용
- web scaping은 BeautifulSoup4 라이브러리를 사용하세요.
- 데이터 처리를 위해서 pandas 라이브러리를 사용하세요.
- 로그 기록 시에 datetime 라이브러리를 사용하세요.

##### 코드 가독성
- 코드 가독성을 높이기 위해 1) 주석을 사용해서 설명을 추가하고 2) 함수를 만들어서 가독성과 재사용성을 높이세요.

##### 화일 포맷, 데이터베이스 이름 등
- 추출 (Extract)한 정보는 'Countries_by_GDP.json'라는 이름의 JSON 화일 포맷으로 저장해야 합니다.
- 필요한 모든 작업을 수행하는 'etl_project_gdp.py' 코드를 작성하세요.

##### ETL 프로세스
- ETL 프로세스에 따라 코드를 작성하고 각 단계의 시작과 끝을 로그에 기록하세요.
- 이 모든 처리 과정은 'etl_project_log.txt'라는 로그 화일에 기록되어야 합니다. (로그 화일은 매번 다시 생성하는 것이 아니라 기존 화일에 append되어야 합니다.)
- log는 "time, log" 형식으로 기록하세요. 시간은 'Year-Monthname-Day-Hour-Minute-Second' 포맷에 맞게 표시하세요.



#### 팀 활동 요구사항
- wikipeida 페이지가 아닌, IMF 홈페이지에서 직접 데이터를 가져오는 방법은 없을까요? 어떻게 하면 될까요?
- 만약 데이터가 갱신되면 과거의 데이터는 어떻게 되어야 할까요? 과거의 데이터를 조회하는 게 필요하다면 ETL 프로세스를 어떻게 변경해야 할까요?



#### 추가 요구 사항
##### 코드를 수정해서 아래 요구사항을 구현하세요.
- 추출한 데이터를 데이터베이스에 저장하세요. 'Countries_by_GDP'라는 테이블명으로 'World_Economies.db'라는 데이터 베이스에 저장되어야 합니다. 해당 테이블은 'Country', 'GDP_USD_billion'라는 어트리뷰트를 반드시 가져야 합니다.
    - 데이터베이스는 sqlite3 라이브러리를 사용해서 만드세요.
- 필요한 모든 작업을 수행하는 'etl_project_gdp_with_sql.py' 코드를 작성하세요.

##### 화면 출력
- SQL Query를 사용해야 합니다.
- GDP가 100B USD이상이 되는 국가만을 구해서 화면에 출력해야 합니다.
- 각 Region별로 top5 국가의 GDP 평균을 구해서 화면에 출력해야 합니다.

---

### 추가 요구사항 적용 X 

### version1: 출력해야하는 함수를 따로 만들지 않고 Transform 단계에 넣은 것

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import json

# 로그 기록 함수
def log_message(message):
    timestamp = datetime.datetime.now().strftime('%Y-%b-%d-%H-%M-%S')
    with open('etl_project_log.txt', 'a') as log_file:
        log_file.write(f"{timestamp}, {message}\n")

# Extract
def extract_gdp_data(url):
    log_message("Starting Data Extraction")
    # HTTP 요청 후 파싱해서 데이터 추출
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'class': 'wikitable'})
    rows = table.find_all('tr')

    data = []
    for row in rows[1:]:  # 첫 번째 행은 헤더
        cols = row.find_all('td')
        cleaned_cols = []
        for col in cols:
            # 'sup' 태그 제거
            for sup in col.find_all('sup'):
                sup.decompose()
            cleaned_cols.append(col.text.strip())  # 텍스트 정리
        if cleaned_cols:  # 빈 행 제외
            try:
                country = cleaned_cols[0]  # 국가명
                gdp_raw = cleaned_cols[1]  # GDP 값 (Nominal GDP)
                gdp_year = cleaned_cols[2]
                gdp_cleaned = ''.join(filter(str.isdigit, gdp_raw.split('.')[0]))
                if gdp_cleaned:  # GDP 값이 유효한 경우만 추가
                    gdp = int(gdp_cleaned) / 1e3  # 단위를 1B USD로 변환
                    data.append({'Country': country, 'GDP (B USD)': round(gdp, 2), 'Year': gdp_year})
            except IndexError:
                # 예상치 못한 데이터 구조를 무시
                continue
    log_message("Data Extraction Completed")
    
    return data

# Transform
def transform_gdp_data(data):
    log_message("Starting Data Transformation")
    # 데이터프레임 생성
    df = pd.DataFrame(data)

    # GDP 순으로 정렬
    df = df.sort_values(by='GDP (B USD)', ascending=False)
    
    # 국가별 Region 정보 매핑
    with open('country_region_table.json', 'r', encoding='utf-8') as region_file:
        region_data = json.load(region_file)
    df['Region'] = df['Country'].map(region_data)
    
    # GDP가 100B USD 이상인 국가만 필터링
    filtered_df = df[df['GDP (B USD)'] >= 100]
    print(filtered_df)
    
    # Region별 상위 5개 국가의 GDP 평균 계산
    region_top5_avg = (
        filtered_df.groupby('Region')
        .apply(lambda x: x.nlargest(5, 'GDP (B USD)')['GDP (B USD)'].mean())
        .reset_index(name='Top 5 Avg GDP (B USD)')
    )
    
    # 결과 출력
    print(region_top5_avg)
    
    log_message("Data Transformation Completed")
    return df

# Load
def load_gdp_data(df, output_csv_file='gdp_by_country.csv', output_json_file='Countries_by_GDP.json'):
    log_message("Starting Data Loading")
    try:
        # CSV 파일로 저장
        df.to_csv(output_csv_file, index=False)
        log_message(f"CSV file saved as {output_csv_file}")
        
        # JSON 파일로 저장
        data_as_dict = df.to_dict(orient='records')  # DataFrame을 딕셔너리 리스트로 변환
        with open(output_json_file, 'w', encoding='utf-8') as json_file:
            json.dump(data_as_dict, json_file, ensure_ascii=False, indent=4)
        log_message(f"JSON file saved as {output_json_file}")
        
        log_message("Data Loading Completed Successfully")
        
    except Exception as e:
        log_message(f"Data Loading Failed: {str(e)}")
        raise  # 예외를 다시 던져 ETL 프로세스에서 처리 가능

# 메인 ETL 함수
def etl_process():
    url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"
    try:
        log_message("ETL Process Started")
        # Extract
        data = extract_gdp_data(url)
        # Transform
        transformed_data = transform_gdp_data(data)
        # Load
        load_gdp_data(transformed_data)  
        log_message("ETL Process End Successfully")
        
    except Exception as e:
        log_message(f"ETL Process Failed: {str(e)}")

if __name__ == "__main__":
    etl_process()

/Users/admin/Documents/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


          Country  GDP (B USD)  Year         Region
0           World    115494.31  2025            NaN
1   United States     30337.16  2025  North America
2           China     19534.89  2025           Asia
3         Germany      4921.56  2025         Europe
4           Japan      4389.33  2025           Asia
..            ...          ...   ...            ...
68     Uzbekistan       112.65  2024           Asia
69      Guatemala       112.37  2024  North America
70           Oman       109.99  2024           Asia
71       Bulgaria       108.42  2024         Europe
72      Venezuela       106.33  2024  South America

[73 rows x 4 columns]
          Region  Top 5 Avg GDP (B USD)
0         Africa             238.182000
1           Asia            6255.970000
2         Europe            3318.112000
3  North America            6946.500000
4        Oceania            1212.226667
5  South America             791.566000


/var/folders/rd/d9knhb6x6nj7rd817gqlcsr00000gn/T/ipykernel_5634/3578079735.py:67: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_df.groupby('Region')


### version2: 출력해야하는 것을 함수로 따로 빼놓은 것

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import json

# 로그 기록 함수
def log_message(message):
    timestamp = datetime.datetime.now().strftime('%Y-%b-%d-%H-%M-%S')
    with open('etl_project_log.txt', 'a') as log_file:
        log_file.write(f"{timestamp}, {message}\n")

# Extract
def extract_gdp_data(url):
    log_message("Starting Data Extraction")
    # HTTP 요청 후 파싱해서 데이터 추출
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'class': 'wikitable'})
    rows = table.find_all('tr')

    data = []
    for row in rows[1:]:  # 첫 번째 행은 헤더이므로 제외함
        cols = row.find_all('td')
        cleaned_cols = []
        for col in cols:
            # 'sup' 태그 제거 [n 1] 이런식으로 자꾸 떠서
            for sup in col.find_all('sup'):
                sup.decompose()
            cleaned_cols.append(col.text.strip())  # 텍스트 정리
        if cleaned_cols:  # 빈 행 제외
            try:
                country = cleaned_cols[0]  # 국가명
                gdp_raw = cleaned_cols[1]  # GDP 값 (Nominal GDP)
                gdp_year = cleaned_cols[2]
                gdp_cleaned = ''.join(filter(str.isdigit, gdp_raw.split('.')[0]))
                if gdp_cleaned:  # GDP 값이 유효한 경우만 추가
                    gdp = int(gdp_cleaned) / 1e3  # 단위를 1B USD로 변환
                    data.append({'Country': country, 'GDP (B USD)': round(gdp, 2), 'Year': gdp_year})
            except IndexError:
                # 예상치 못한 데이터 구조를 무시
                continue
    log_message("Data Extraction Completed")
    
    return data

# Transform
def transform_gdp_data(data):
    log_message("Starting Data Transformation")
    # 데이터프레임 생성
    df = pd.DataFrame(data)

    # GDP 순으로 정렬
    df = df.sort_values(by='GDP (B USD)', ascending=False)
    
    # 국가별 Region 정보 매핑
    with open('country_region_table.json', 'r', encoding='utf-8') as region_file:
        region_data = json.load(region_file)
    df['Region'] = df['Country'].map(region_data)
    
    log_message("Data Transformation Completed")
    return df

# Load
def load_gdp_data(df, output_csv_file='gdp_by_country.csv', output_json_file='Countries_by_GDP.json'):
    log_message("Starting Data Loading")
    try:
        # CSV 파일로 저장
        df.to_csv(output_csv_file, index=False)
        log_message(f"CSV file saved as {output_csv_file}")
        
        # JSON 파일로 저장
        data_as_dict = df.to_dict(orient='records')  # DataFrame을 딕셔너리 리스트로 변환
        with open(output_json_file, 'w', encoding='utf-8') as json_file:
            json.dump(data_as_dict, json_file, ensure_ascii=False, indent=4)
        log_message(f"JSON file saved as {output_json_file}")
        
        log_message("Data Loading Completed Successfully")
        
    except Exception as e:
        log_message(f"Data Loading Failed: {str(e)}")
        raise  # 예외를 다시 던져 ETL 프로세스에서 처리 가능


# GDP가 100B USD 이상인 국가만 필터링
def filtered_100USD(df):
    filtered_100 = df[df['GDP (B USD)'] >= 100]
    print("Countries with a GDP of over 100B USD")
    print(filtered_100)
    
    
 # Region별 상위 5개 국가의 GDP 평균 계산
def region_top5_calculate(df):
    region_top5_avg = (
        df.groupby('Region')
        .apply(lambda x: x.nlargest(5, 'GDP (B USD)')['GDP (B USD)'].mean())
        .reset_index(name='Top 5 Avg GDP (B USD)')
    )
    print("Average GDP of top 5 countries by region")
    print(region_top5_avg)
    

# 메인 ETL 함수
def etl_process():
    url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"
    try:
        log_message("ETL Process Started")
        # Extract
        data = extract_gdp_data(url)
        # Transform
        transformed_data = transform_gdp_data(data)
        # Load
        load_gdp_data(transformed_data)  
        log_message("ETL Process End Successfully")
        
        # GDP가 100B USD 이상인 국가만 필터링
        filtered_100USD(transformed_data)
        # Region별 상위 5개 국가의 GDP 평균 계산
        region_top5_calculate(transformed_data)
        
    except Exception as e:
        log_message(f"ETL Process Failed: {str(e)}")

if __name__ == "__main__":
    etl_process()

Countries with a GDP of over 100B USD
          Country  GDP (B USD)  Year         Region
0           World    115494.31  2025            NaN
1   United States     30337.16  2025  North America
2           China     19534.89  2025           Asia
3         Germany      4921.56  2025         Europe
4           Japan      4389.33  2025           Asia
..            ...          ...   ...            ...
68     Uzbekistan       112.65  2024           Asia
69      Guatemala       112.37  2024  North America
70           Oman       109.99  2024           Asia
71       Bulgaria       108.42  2024         Europe
72      Venezuela       106.33  2024  South America

[73 rows x 4 columns]
Average GDP of top 5 countries by region
          Region  Top 5 Avg GDP (B USD)
0         Africa                238.182
1           Asia               6255.970
2         Europe               3318.112
3  North America               6946.500
4        Oceania                734.840
5  South America                79

/var/folders/rd/d9knhb6x6nj7rd817gqlcsr00000gn/T/ipykernel_5634/705097223.py:95: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('Region')


#### 팀 활동 요구사항

- IMF 홈페이지에서 가져오기 : https://www.imf.org/en/Publications/WEO/weo-database/2024/October/weo-report?c=512,914,612,171,614,311,213,911,314,193,122,912,313,419,513,316,913,124,339,638,514,218,963,616,223,516,918,748,618,624,522,622,156,626,628,228,924,233,632,636,634,238,662,960,423,935,128,611,321,243,248,469,253,642,643,939,734,644,819,172,132,646,648,915,134,652,174,328,258,656,654,336,263,268,532,944,176,534,536,429,433,178,436,136,343,158,439,916,664,826,542,967,443,917,544,941,446,666,668,672,946,137,546,674,676,548,556,678,181,867,682,684,273,868,921,948,943,686,688,518,728,836,558,138,196,278,692,694,962,142,449,564,565,283,853,288,293,566,964,182,359,453,968,922,714,862,135,716,456,722,942,718,724,576,936,961,813,726,199,733,184,524,361,362,364,732,366,144,146,463,528,923,738,578,537,742,866,369,744,186,925,869,746,926,466,112,111,298,927,846,299,582,487,474,754,698,&s=NGDPD,&sy=2022&ey=2029&ssm=0&scsm=1&scc=0&ssd=1&ssc=0&sic=0&sort=country&ds=.&br=1

- 덤프파일이나 API?

- 과거의 데이터는 칼럼에 연도별로 추가해보기?
- row가 많지 않으니까 칼럼을 쌓는 방법도 있는 듯
- 

### 생각해보기

#### 1. raw 데이터의 양이 압도적으로 많다면?
- 데이터 이동 시 네트워크 병목 발생 가능하겠지..?
- BeautifulSoap가 아닌 다른 라이브러리를 사용해볼까? 
- 분산 컴퓨팅 도구를 쓴다면? (그렇다면 어떤 도구? 스파크?)
- 클라우드 서비스를 사용? 비용은?
- ELT가 아닌 ETL로 하면 어떨까? 무슨 다른 문제점이 발생할까?
-

#### 2. raw 데이터를 Transform 하는데 시간이 아주 오래 걸린다면?
- Transform의 단계를 더 세분화 하면 어떨까? 어떻게 할까?
- Waterfall처럼 단계를 저장하면서 한다면? 오히려 저장하는 빈도 및 양이 늘어나서 비효율적일까..?
- 자주 쓰는 데이터는 저장하여 캐싱을 해볼까?


---

## Transform 에서 추출한 정보를 json으로 저장하고, Load에서 sqlite로 저장해보기